In [2]:
import os
from typing import Any

from dotenv import load_dotenv
from utils.agent_visualizer import print_activity

from claude_code_sdk import ClaudeCodeOptions, ClaudeSDKClient

# 02 - The Observability Agent

In the previous notebooks we have built a basic research agent and a Chief of Staff multi-agent framework. While the agents we have built are already powerful, they were still limited in what they could do: the web search agent is limited to searching the internet and our Chief of Staff agent was limited to interacting with its own filesystem.

This is a serious constraint: real-world agents often need to interact with other systems like databases, APIs, file systems, and other specialized services. [MCP (Model Context Protocol)](https://modelcontextprotocol.io/docs/getting-started/intro) is an open-source standard for AI-tool integrations that allows for an easy connection between our agents and these external systems. In this notebook, we will explore how to connect MCP servers to our agent.

**Need more details on MCP?** For comprehensive setup instructions, configuration best practices, and troubleshooting tips, see the [Claude Code MCP documentation](https://docs.claude.com/en/docs/claude-code/mcp).

## Introduction to the MCP Server
### 1. The Git MCP server

Let's first give our agent the ability to understand and work with Git repositories. By adding the [Git MCP server](https://github.com/modelcontextprotocol/servers/tree/main/src/git) to our agent, it gains access to 13 Git-specific tools that let it examine commit history, check file changes, create branches, and even make commits. This transforms our agent from a passive observer into an active participant in your development workflow. In this example, we'll configure the agent to explore a repository's history using only Git tools. This is pretty simple, but knowing this, it is not difficult to imagine agents that can automatically create pull requests, analyze code evolution patterns, or help manage complex Git workflows across multiple repositories.

In [ ]:
# define our git MCP server (it was downloaded when you ran uv sync as it is defined in the pyproject.toml file)
git_mcp: dict[str, Any] = {
    "git": {
        "command": "uv",
        "args": ["run", "python", "-m", "mcp_server_git", "--repository", os.getcwd()],
    }
}

In [4]:
messages = []
async with (
    ClaudeSDKClient(
        options=ClaudeCodeOptions(
            model="claude-sonnet-4-20250514",
            mcp_servers=git_mcp,
            allowed_tools=[
                "mcp__git"
            ],  # For MCP tools, in allowed tools we must add the mcp__serverName__toolName format or mcp__serverName to enable all
            permission_mode="acceptEdits",  # auto-accept file edit permissions
        )
    ) as agent
):
    await agent.query(
        "Use ONLY your git mcp tools to quickly explore this repo's history and gimme a brief summary."
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: mcp__git()
✓ Tool completed
🤖 Thinking...
🤖 Using: Bash()
🤖 Using: Bash()
🤖 Using: Bash()
✓ Tool completed
✓ Tool completed
✓ Tool completed
🤖 Thinking...


In [ ]:
print(f"\nResult:\n{messages[-1].result}")

### 2. The GitHub MCP server

Now let's level up from local Git operations to full GitHub platform integration. By switching to the [official GitHub MCP server](https://github.com/github/github-mcp-server/tree/main), our agent gains access to over 100 tools that interact with GitHub's entire ecosystem – from managing issues and pull requests to monitoring CI/CD workflows and analyzing code security alerts. This server can work with both public and private repositories, giving your agent the ability to automate complex GitHub workflows that would typically require multiple manual steps.

#### Step 1: Set up your GitHub Token

You need a GitHub Personal Access Token. Get one [here](https://github.com/settings/personal-access-tokens/new) and put in the .env file as ```GITHUB_TOKEN="<token>"```
> Note: When getting your token, select "Fine-grained" token with the default options (i.e., public repos, no account permissions), that'll be the easiest way to get this demo working.

Also, for this example you will have to have [Docker](https://www.docker.com/products/docker-desktop/) running on your machine. Docker is required because the GitHub MCP server runs in a containerized environment for security and isolation.

**Docker Quick Setup:**
- Install Docker Desktop from [docker.com](https://www.docker.com/products/docker-desktop/)
- Ensure Docker is running (you'll see the Docker icon in your system tray)
- Verify with `docker --version` in your terminal
- **Troubleshooting:** If Docker won't start, check that virtualization is enabled in your BIOS. For detailed setup instructions, see the [Docker documentation](https://docs.docker.com/get-docker/)

#### Step 2: Define the mcp server and start the agent loop!

In [6]:
# define our github mcp server
load_dotenv(override=True)
github_mcp: dict[str, Any] = {
    "github": {
        "command": "docker",
        "args": [
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server",
        ],
        "env": {"GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_TOKEN")},
    }
}

In [7]:
# run our agent
messages = []
async with ClaudeSDKClient(
    options=ClaudeCodeOptions(
        model="claude-sonnet-4-20250514",
        mcp_servers=github_mcp,
        allowed_tools=["mcp__github"],
        permission_mode="acceptEdits",  # auto-accept permissions
    )
) as agent:
    await agent.query(
        "Use ONLY your GitHub MCP tools to search for the anthropics/claude-code-sdk-python repository and and give me a couple facts about it"
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: mcp__github__search_repositories()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_file_contents()
✓ Tool completed
🤖 Thinking...


In [ ]:
print(f"\nResult:\n{messages[-1].result}")

## Real use case: An observability agent

Now, with such simple setup we can already have an agent acting as self-healing software system!

In [9]:
load_dotenv(override=True)

prompt = """Monitor the GitHub Actions workflows for facebook/react.
Look at the last triggered CI pipeline. 
1. Analyze the trigger for the pipeline
2. Identify whether the pipeline passed or not
3. If it failed, explain which test failed
4. Identify whether human involvement is required

IMPORTANT: Do not raise a PR, issue, or bug on github yet. Just give me a summary of your findings and plan.

Focus on the 'CI' workflow specifically. Use your Github MCP server tools!"""

github_mcp: dict[str, Any] = {
    "github": {
        "command": "docker",
        "args": [
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server",
        ],
        "env": {"GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_TOKEN")},
    }
}

messages = []
async with ClaudeSDKClient(
    options=ClaudeCodeOptions(
        model="claude-sonnet-4-20250514",
        mcp_servers=github_mcp,
        allowed_tools=["mcp__github"],
        permission_mode="acceptEdits",
    )
) as agent:
    await agent.query(prompt)
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__list_workflows()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__list_workflow_runs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_workflow_run()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__list_workflow_jobs()
✓ Tool completed
🤖 Thinking...
🤖 Using: WebFetch()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...


In [ ]:
print(f"\nResult:\n{messages[-1].result}")

### Observability Agent as Module

The `observability_agent/agent.py` file contains the same minimal helper functions as the research agent or chief of staff agent, just enhanced for GitHub monitoring. 

As before, to use it as a module in your Python code:

In [ ]:
from observability_agent.agent import send_query

result = await send_query(
    "Check the CI status for the last 2 runs in anthropics/claude-code-sdk-python. Just do 3 tool calls, be efficient."
)
print(f"Monitoring result: {result}")

We can do multi-turn conversations with this agent as well:

In [ ]:
# Example 2: Multi-turn conversation for deeper monitoring
result1 = await send_query("What's the current CI status for facebook/react?")
print(f"Initial check: {result1[:250]}...\n")

In [ ]:
# Continue the conversation to dig deeper
result2 = await send_query(
    "Are there any flaky tests in the recent failures? You can only make one tool call.",
    continue_conversation=True,
)
print(f"Follow-up analysis: {result2[:250]}...")

## Conclusion

We've demonstrated how the Claude Code SDK enables seamless integration with external systems through the Model Context Protocol (MCP). Starting with local Git operations through the Git MCP server, we progressively expanded to full GitHub platform integration with access to over 100 GitHub-specific tools. This transformed our agent from a local assistant into a powerful observability system capable of monitoring workflows, analyzing CI/CD failures, and providing actionable insights for production systems.

By connecting MCP servers to our agent, we created an autonomous observability system that monitors GitHub Actions workflows, distinguishes between real failures and security restrictions, and provides detailed analysis of test failures. The system demonstrates how agents can actively participate in your DevOps workflow, moving from passive monitoring to intelligent incident response.

This concludes, for now, our journey through the Claude Code SDK tutorial series. We've progressed from simple research agents to sophisticated multi-agent orchestration, and finally to external system integration through MCP. Together, these patterns provide the foundation for building production-ready agentic systems that can handle real-world complexity while maintaining governance, compliance, and observability.

### What You've Learned Across All Notebooks

**From Notebook 00 (Research Agent)**
- Core SDK fundamentals with `query()` and `ClaudeSDKClient`
- Basic tool usage with WebSearch and Read
- Simple agent loops and conversation management

**From Notebook 01 (Chief of Staff)**
- Advanced features: memory, output styles, planning mode
- Multi-agent coordination through subagents
- Governance through hooks and custom commands
- Enterprise-ready agent architectures

**From Notebook 02 (Observability Agent)**
- External system integration via MCP servers
- Real-time monitoring and incident response
- Production workflow automation
- Scalable agent deployment patterns

The complete implementations for all three agents are available in their respective directories (`research_agent/`, `chief_of_staff_agent/`, `observability_agent/`), ready to serve as inspiration for integrations into your production systems.